In [10]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json


Define URLs:

In [137]:
ls_2014 = "http://myneta.info/ls2014/index.php?action=show_winners&sort=default"
ls_2009 = "http://myneta.info/ls2009/index.php?action=show_winners&sort=default"
ls_2004 = "http://myneta.info/loksabha2004/index.php?action=show_winners&sort=default"

Define function to download and create dataframe

In [138]:
def process_link(link, year):
    
    html = requests.get(link)
    doc = pq(html.content)
    trs = doc('table').eq(2).children('tr')

    datalist=[]

    for tr in trs:
        winner_dict = {'year':year}
        for i,td in enumerate(tr.findall('td')):
            if i == 0:
                winner_dict['idnum'] = td.text
            if i == 1:
                winner_dict['name'] = td.findall('a')[1].text
            if i == 2:
                winner_dict['constituency'] = td.text
            if i == 3:
                winner_dict['party'] = td.text
            if i == 4:
                winner_dict['crim_cases'] = td.text
                if td.text == None:
                    winner_dict['crim_cases'] = td.find('span').text
            if i == 5:
                winner_dict['education'] = td.text
            if i == 6:
                winner_dict['assets'] =  td.text.replace(',','').replace('Rs','').lstrip()
            if i == 7:
                winner_dict['liabilities'] =  td.text.replace(',','').replace('Rs','').lstrip()

        datalist.append(winner_dict)

    return pd.DataFrame(datalist)
    

In [143]:
df_14 = process_link(ls_2014, 2014)
df_09 = process_link(ls_2009, 2009)
df_04 = process_link(ls_2004, 2004)
df_master = pd.concat([df_14,df_09,df_04])
df_master.to_csv('loksabha_win.csv')